## FEWS NET Data Warehouse (FDW) Crop Data Preparation
This script accesses and obtains crop production data from [FEWS NET Data Warehouse (FDW)](https://fdw.fews.net/en/).
- Full API document is [here](https://fdw.fews.net/en/docs/users/api.html).
- FEWS NET country administrative boundaries are [here](https://fews.net/fews-data/334).
- [FAO - Food Security and Nutrition Analysis Unit - Somalia](https://fsnau.org/)

By Donghoon Lee @ 12-03-2020

In [1]:
import sys
import warnings
from itertools import product
from functools import reduce
from configparser import ConfigParser
from io import BytesIO, StringIO
import requests
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import geopandas as gpd
import json
import seaborn as sns
from tools import save_hdf, cbarpam
# Mapping
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import ListedColormap
import seaborn as sns
sns.set_style("white", {'axes.linewidth': 1, 'grid.color': 'black'})

# Function to invert a dictionary
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = key 
            else: 
                inverse[item].append(key) 
    return inverse

### Retreive crop production indicator value from FDW

In [2]:
# Host address
host = 'https://fdw.fews.net'
auth = ('dlee', 'fdwdonghoon')
# Crop list
product_list = {'R01142AA': 'Sorghum',                  # KE, SO
                'R01142AC': 'Maize Grain (White)',      # KE
                'R01122AA': 'Maize (Corn)'}             # SO
# Set up the parameters
parameters = {'format': 'json',
              'country': ['Somalia', 'Kenya'],
              'survey_type': 'crop:best',
              'product': ['R01122AA','R01122AC']}
# Get the Crop Production Indicator Values (i.e. the list of Data Points)
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
# Turn the records into a Pandas DataFrame
df = pd.DataFrame.from_records(response.json())
retain_cols = ['country','fnid','admin_1','admin_2',
               'population_group', 'crop_production_system',
               'start_date','period_date',
               'season_name','season_type','season_date','season_year',
               'indicator','value','locality_name'] 
df = df[retain_cols]
df = df.drop_duplicates().reset_index(drop=True)
df_Kenya = df[df.country == 'Kenya']
df_Somalia = df[df.country == 'Somalia']

## 1) Kenya
Kenya crop seasonal calendar </br>
![FEWS NET](https://fews.net/sites/default/files/styles/large/public/seasonal-calendar-kenya.png?itok=UpLuf1Bl)

### (1) Relationships between administrative units
- In Kenya, basd on the FEWS NET data, there were four times of changes in administrative units: 1982, 1989, 2009, and 2013.

| Year | Admin-1 | # units  | Admin-2  | # units |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1982 | KE1982A1| 8        | KE1982A2 | 41      |
| 1989 | KE1989A1| 8        | KE1989A2 | 47      |
| 2009 | KE2009A1| 8        | KE2009A2 | 47      |
| 2013 | **`KE2013A1`**| 47 | KE2013A2 | 270     |
- In 1989, 6 districts are divided and added: Bomet, Makueni, Migori, Nyamira, Tharaka Nithi, and Vihiga.

| 1982-1989 (original) |1989-2013 (changed)|1989-2013 (added)|
| :---:| :---:|:---:|
|Meru (KE1982A25020)    | Meru (KE1989A25017)    | Tharaka Nithi (KE1989A25018)|
|Machakos (KE1982A25019)| Machakos(KE1989A25004) | Makueni       (KE1989A25005)|
|Kericho  (KE1982A25424)| Kericho  (KE1989A25422)| Bomet         (KE1989A25420)|
|Kissi    (KE1982A25319)| Kissi    (KE1989A25314)| Nyamira       (KE1989A25308)|
|Homa Bay (KE1982A25318)| Homa Bay (KE1989A25313)| Migori        (KE1989A25316)|
|Kakamega (KE1982A25512)| Kakamega (KE1989A25511)| Vihiga        (KE1989A25508)|

- In 2013, Admin-2 became Admin-1 (No name changes).
- Here, we use **`KE2013A1`** to represent all admin-level 1 crop data.

In [3]:
# Load FEWS NET shapfiles
KE_adm1_2013 = gpd.read_file('./data/shapefile/KE_Admin1_2013.shp')
KE_adm1_2013_name = KE_adm1_2013['ADMIN1'].values
KE_adm2_1982 = gpd.read_file('./data/shapefile/KE_Admin2_1982.shp')
KE_adm2_1982_name = KE_adm2_1982['ADMIN2'].values
KE_adm2_1989 = gpd.read_file('./data/shapefile/KE_Admin2_1989.shp')
KE_adm2_1989_name = KE_adm2_1989['ADMIN2'].values
ADM1_LIST = list(KE_adm1_2013.FNID)

# 1) Changes in 1989: New 6 districts are splitted/added 
#    : ['Bomet', 'Makueni', 'Migori', 'Nyamira', 'Tharaka Nithi', 'Vihiga']
# KE_adm2_1989_name[~np.isin(KE_adm2_1989_name, KE_adm2_1982_name)]
# Meru     (KE1982A25020) -> Meru     (KE1989A25017) and Tharaka Nithi (KE1989A25018)
# Machakos (KE1982A25019) -> Machakos (KE1989A25004) and Makueni       (KE1989A25005)
# Kericho  (KE1982A25424) -> Kericho  (KE1989A25422) and Bomet         (KE1989A25420)
# Kissi    (KE1982A25319) -> Kissi    (KE1989A25314) and Nyamira       (KE1989A25308)
# Homa Bay (KE1982A25318) -> Homa Bay (KE1989A25313) and Migori        (KE1989A25316)
# Kakamega (KE1982A25512) -> Kakamega (KE1989A25511) and Vihiga        (KE1989A25508)
adm_split1 = {'KE1982A25020':['KE1989A25017','KE1989A25018'],
              'KE1982A25019':['KE1989A25004','KE1989A25005'],
              'KE1982A25424':['KE1989A25422','KE1989A25420'],
              'KE1982A25319':['KE1989A25314','KE1989A25308'],
              'KE1982A25318':['KE1989A25313','KE1989A25316'],
              'KE1982A25512':['KE1989A25511','KE1989A25508']}
adm_split1 = invert_dict(adm_split1)

### The second link is deprecated.
# # 2) Only two adm-1 data exist before 2013: Nyanza and Nairobi
# # Since FDW doesn't have adm-2 level for Nairobi, we can just link Nyanza.
# adm_split2 = {'KE1982A153': ['KE1989A25313','KE1989A25314','KE1989A25315','KE1989A25316','KE1989A25308','KE1989A25317']}
# adm_split2 = invert_dict(adm_split2)

# 3) In 2013, adm-2 became adm-1 (No name changes)
assert all(np.sort(KE_adm1_2013_name) == np.sort(KE_adm2_1989_name)) == True

### (2) Generate base tables of Area and Production

In [4]:
# Generate an initial data
fnid_name = KE_adm1_2013[['FNID','ADMIN1']]
df = df_Kenya.copy()

# Loop by Name (we confirmed that names are the same)
frame_area = []
frame_prod = []
for i, (fnid_org, name_org) in fnid_name.iterrows():
    # Admin level 1
    subdf = df[df.admin_1 == name_org]
    time = pd.to_datetime(subdf['season_date'].values)
    # Check if data has more than 3 records in a year
    assert (np.unique(time,return_counts=True)[1] > 3).sum() == 0
    pivot_adm1 = pd.pivot_table(subdf, values='value', index='season_date', columns=['indicator'], aggfunc=sum)
    # Admin level 2
    subdf = df[df.admin_2 == name_org]
    time = pd.to_datetime(subdf['season_date'].values)
    # Check if data has more than 4 season records in a year
    assert (np.unique(time,return_counts=True)[1] > 3).sum() == 0
    pivot_adm2 = pd.pivot_table(subdf, values='value', index='season_date', columns=['indicator'], aggfunc=sum)
    # Base pivot table
    pivot = pd.concat((pivot_adm2, pivot_adm1), axis=0)
    # Remove season if either area and production is missing
    pivot = pivot[pivot.isna().sum(1) == 0]

    # Fill missing annual data with available short/long data
    # - if both short/long seasons exist, we do summation
    # - if a single season exists, we just copy it as annual data
    time = pd.to_datetime(pivot.index)
    pivot['year'], pivot['month'] = time.year, time.month
    pivot_sub = pivot[pivot.month != 10].groupby('year').sum().reset_index(drop=False)
    pivot_sub['month'], pivot_sub['day'] = 10, 1
    pivot_sub.index = pd.to_datetime(pivot_sub[['year', 'month','day']]).dt.strftime('%Y-%m-%d')
    pivot_sub.index.name = 'season_date'
    pivot_sub = pivot_sub[['Area Planted','Quantity Produced','Yield','year','month']]
    pivot = pd.concat((pivot, pivot_sub),axis=0).sort_index()

    # Final area and production tables
    table_area = pd.pivot_table(pivot, index='year',columns='month',values='Area Planted')
    table_area = pd.DataFrame(data=table_area, index=table_area.index, columns=[2,7,10])
    table_prod = pd.pivot_table(pivot, index='year',columns='month',values='Quantity Produced')
    table_prod = pd.DataFrame(data=table_prod, index=table_prod.index, columns=[2,7,10])

    # MultiIndex
    tuples = [(fnid_org, name_org, 'Short', '03-31'),
              (fnid_org, name_org, 'Long', '08-31'),
              (fnid_org, name_org, 'Annual', '01-31')]
    mdx = pd.MultiIndex.from_tuples(tuples, names=['FNID','name','season','harvest'])
    table_area.columns = mdx
    table_prod.columns = mdx
    frame_area.append(table_area)
    frame_prod.append(table_prod)
area = pd.concat(frame_area, axis=1)
prod = pd.concat(frame_prod, axis=1)

### (3) Import external datasets
- "National Maize Production 2014 - 2019.xlsx"
- "KE_FEWS_Production data up to_Apr2015.xlsx"

These datasets are prepared by Liya Manga (FEWS NET) and slightly modified (mostly removing unrealistic ticks) by Donghoon Lee.

In [5]:
# (a) Import data table of "National Maize Production 2014 - 2019.xlsx"
fn = './data/crop/forwarded/National Maize Production 2014 - 2019_modified_dlee.xlsx'
df = pd.read_excel(fn, sheet_name='modified_by_dlee', header=[0,1,2,3], index_col=0)
# Import 2017-2019 (FDW has data until 2016)
df = df.loc(axis=1)[pd.IndexSlice[np.arange(2017,2020),:,:,:]]
# Convert the number of 90kg bags to tons
df.loc(axis=1)[pd.IndexSlice[:,:,'Production',:]] = df.loc(axis=1)[pd.IndexSlice[:,:,'Production',:]] * 90/1000
df.columns = df.columns.droplevel(3)
# Insert area and production
for em in product([2017,2018,2019],['Long','Short'],['Area','Production']):
    year, season, indicator = em[0], em[1], em[2]
    sdata = df[(year,season,indicator)]
    for name, value in sdata.iteritems():
        if indicator == 'Area':
            area.loc[year,pd.IndexSlice[:,name,season,:]] = value
        elif indicator == 'Production':
            prod.loc[year,pd.IndexSlice[:,name,season,:]] = value
# Aggregate annual values
for year in [2017,2018,2019]:
    area_annual = np.nansum([area.loc[year,pd.IndexSlice[:,:,['Short'],:]].values,
                             area.loc[year,pd.IndexSlice[:,:,['Long'],:]].values],axis=0)
    area.loc[year,pd.IndexSlice[:,:,['Annual'],:]] = area_annual
    prod_annual = np.nansum([prod.loc[year,pd.IndexSlice[:,:,['Short'],:]].values,
                             prod.loc[year,pd.IndexSlice[:,:,['Long'],:]].values],axis=0)
    prod.loc[year,pd.IndexSlice[:,:,['Annual'],:]] = prod_annual

    
# (b) Import data table of "KE_FEWS_Production data up to_Apr2015.xlsx"
fn = './data/crop/forwarded/KE_FEWS_Production data up to_Apr2015_modified_dlee.xlsx'
df = pd.read_excel(fn, sheet_name='modified_by_dlee', header=[0,1], index_col=0)
df[df == 0] = np.nan
# Remove years with all missing records
df = df[~(df.isna().sum(1) == df.shape[1])]
names = df.columns.get_level_values(0).unique()
# Extend time period of base tables
area = area.reindex(np.arange(df.index.min(), area.index.max()+1))
prod = prod.reindex(np.arange(df.index.min(), prod.index.max()+1))
for name in names:
    # Find missing years of base tables
    temp = area.loc[:,pd.IndexSlice[:,name,'Annual',:]]
    year_missing = temp.index[temp.isna().values.flatten()]
    year_missing = year_missing[year_missing <= df.index.max()]
    # If the Excel file has a record (both area and production) for the missing year, import it
    for year in year_missing:
        sub = df.loc[year, pd.IndexSlice[name,:]]
        if sub.notna().sum() == 2:
            area.loc[year,pd.IndexSlice[:,name,'Annual',:]] = sub.loc[pd.IndexSlice[name, 'Area']]
            prod.loc[year,pd.IndexSlice[:,name,'Annual',:]] = sub.loc[pd.IndexSlice[name, 'Production']]

### (4) Link yield between original and divided districts
Here, we just copy the yield record (not area and production) of the original districts to the divided districts which are currently missing.

In [6]:
# Calculate yield (yield = production / area)
crop = prod/area
df_raw = df_Kenya.copy()
# Copy historical records of the original district to the divided district
for new, old in adm_split1.items():
    df_raw[df_raw.fnid == old]
    if len(df_raw[df_raw.fnid == old]) > 0:
        year_end = int(df_raw[df_raw.fnid == old].season_date.max()[:4])
        name_new = df_raw[df_raw.fnid == new].admin_2.unique()[0]
        name_old = df_raw[df_raw.fnid == old].admin_2.unique()[0]
        crop.loc[:year_end, pd.IndexSlice[:, name_new ,'Annual',:]] = crop.loc[:year_end, pd.IndexSlice[:, name_old ,'Annual',:]].values

### (5) Finalize yield data

In [7]:
# Re-define names
area1, prod1, crop1 = area, prod, crop

# Calculate national annual yield (annual total production / annual total planted area)
ntnl_annual = pd.DataFrame(index = area1.index, columns = ['Area','Production','Yield','Miss_districts'])
for year in ntnl_annual.index:
    ntnl_annual.loc[year, 'Area'] = area1.loc[year,pd.IndexSlice[:,:,'Annual',:]].sum()
    ntnl_annual.loc[year, 'Production'] = prod1.loc[year,pd.IndexSlice[:,:,'Annual',:]].sum()
    nmiss = area1.loc[year,pd.IndexSlice[:,:,'Annual',:]].isna().sum()
    ntnl_annual.loc[year, 'Miss_districts'] = nmiss / len(ADM1_LIST) *100
ntnl_annual['Yield'] = ntnl_annual['Production'] / ntnl_annual['Area']

# Annual and seasonal (districts and national)
yield1a = crop1.loc[:,pd.IndexSlice[:,:,'Annual',:]]; yield1a.columns = yield1a.columns.droplevel([1,2,3])
area1a = area1.loc[:,pd.IndexSlice[:,:,'Annual',:]]; area1a.columns = area1a.columns.droplevel([1,2,3])
prod1a = prod1.loc[:,pd.IndexSlice[:,:,'Annual',:]]; prod1a.columns = prod1a.columns.droplevel([1,2,3])

# Export to HDF files
# National
save_hdf('./data/crop/KE_maize_adm0_yield_annual.hdf', ntnl_annual)
# Admin-1 level
save_hdf('./data/crop/KE_maize_adm1_yield_annual.hdf', yield1a)
save_hdf('./data/crop/KE_maize_adm1_area_annual.hdf', area1a)
save_hdf('./data/crop/KE_maize_adm1_prod_annual.hdf', prod1a)

# Export to Excel file
fn_out = './data/crop/KE_maize_summary.xlsx'
with pd.ExcelWriter(fn_out, engine='xlsxwriter') as writer:
    # National
    ntnl_annual.to_excel(writer, sheet_name='adm0_yield_annual')
    # Admin-1 level
    yield1a.to_excel(writer, sheet_name='adm1_yield_annual')
    area1a.to_excel(writer, sheet_name='adm1_area_annual')
    prod1a.to_excel(writer, sheet_name='adm1_prod_annual')

    print('%s is saved..' % fn_out)

./data/crop/KE_maize_adm0_yield_annual.hdf is saved.
./data/crop/KE_maize_adm1_yield_annual.hdf is saved.
./data/crop/KE_maize_adm1_area_annual.hdf is saved.
./data/crop/KE_maize_adm1_prod_annual.hdf is saved.
./data/crop/KE_maize_summary.xlsx is saved..


## 2) Somalia

Somalia crop seasonal calendar </br>
![FEWS NET](https://fews.net/sites/default/files/styles/large/public/seasonal-calendar-somalia.png?itok=6LQ1arWc)

### (1) Relationships between administrative units
- In Somalia, based on FEWS NET data, only 1990 administrative units are available.

| Year | Admin-1 | # units | Admin-2  | # units |
| :--- | :----:  | :----:  | :----:   | ---:    |
| 1990 | **`SO1990A1`**| 18      | SO1990A2 | 74      |
- Most of reliable data are admin-2 level, and FDW has few data for admin-1 level. Therefore, we aggregate/merge admin-2 level data to represent admin-1 level.
- Afmadow (SO1990A22802) has two crop reporting units: Afmadow (SO1990R22805) and Afmadow (Hagar) (SO1990R22806). This is confirmed by FEWS NET crop reporting unit (./data/admin_boundaries/SO_Crop_Units_1990.shp). Since SO1990A22802 has only one off-season record, we aggregate SO1990R22805 and SO1990R22806 to replace SO1990A22802.

| District |Crop reporting units|
| :---:|:---:|
|Afmadow (SO1990A22802) | Afmadow (SO1990R22805) and  Hagar (SO1990R22806)|
- Here, we use **`SO1990A1`** to represent all admin-level 1 crop data.

In [8]:
# Load FEWS NET shapfiles
SO_adm1_1990 = gpd.read_file('./data/shapefile/SO_Admin1_1990.shp')   # Total 18 FNID
SO_adm2_1990 = gpd.read_file('./data/shapefile/SO_Admin2_1990.shp')   # Total 74 FNID
ADM1_LIST = SO_adm1_1990.FNID.to_list()
ADM2_LIST = SO_adm2_1990.FNID.to_list()
# Join FNIDs between Admin-1 and Admin-2
join = SO_adm2_1990.merge(SO_adm1_1990[['FNID','ADMIN1']], on='ADMIN1', suffixes=['_adm2', '_adm1'])
join = join[['FNID_adm1','FNID_adm2']]
adm_joint = join.groupby('FNID_adm1')['FNID_adm2'].apply(lambda g: g.values.tolist()).to_dict()

### (2) Generate base tables of Area and Production
- Frank: Issue with post-2010 data reporting separately for Riverine / Agro-pastoral Areas (Kimetrica) – will this affect upload. 
- Annwer: No, it will not affect upload. The livelihood zone is a locality. The FNID, pop_group, and livelihood zone collumns collectivly define the location
- Here, we aggregate both Riverine and Agro-pastoral areas per season to represent seasonal value.

In [9]:
def CreateMultiIndex(fnid, name):
    # MultiIndex
    tuples = [(fnid, name, 'Deyr', '02-28'),
              (fnid, name, 'Deyr-off', '03-31'),
              (fnid, name, 'Gu', '08-31'),
              (fnid, name, 'Gu-off', '10-31'),
              (fnid, name, 'Annual', '12-31')]
    mdx = pd.MultiIndex.from_tuples(tuples, names=['FNID','name','season','harvest'])
    return mdx

# Generate an initial data
df = df_Somalia.copy()
# Name "normal" crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'normal'
# Remove "Area Planted" rows
df = df.loc[~df['indicator'].isin(['Area Planted', 'Yield'])]

# List of FNID and Name 
fnid1_data = df.loc[df.admin_2.isna(), ['fnid','admin_1']].drop_duplicates().reset_index(drop=True)
fnid1_data.columns = ['fnid', 'name']
fnid2_data = df.loc[df.admin_2.notna(), ['fnid','admin_2']].drop_duplicates().reset_index(drop=True)
fnid2_data.columns = ['fnid', 'name']
fnid_data = pd.concat([fnid1_data, fnid2_data],axis=0)

# Loop by FNID
frame_area = []
frame_prod = []
for i, (fnid, name) in fnid_data.iterrows():
    subdf = df[(df.fnid == fnid)].reset_index()
    # Remove data if either "Area Harvested" or "Quantity Produced" is missing in each 
    # "crop_production_system" and "season_date" combination
    comb = subdf[['crop_production_system','season_date']].drop_duplicates()
    for i, (system, season) in comb.iterrows():
        targ = (subdf['crop_production_system'] == system) & (subdf['season_date'] == season)
        temp = subdf.loc[targ, 'value'].values
        if len(temp) != 2 or np.isnan(temp).sum() > 0:
            subdf = subdf[~targ].reset_index(drop=True)
    if len(subdf) == 0: continue        
    # Pivot table
    time = pd.to_datetime(subdf['season_date'].values)
    pivot = pd.pivot_table(subdf, values='value', index='season_date', columns=['indicator'], aggfunc=sum)
    time = pd.to_datetime(pivot.index)
    pivot['year'], pivot['month'] = time.year, time.month
    # Convert to time-series table
    table_area = pd.pivot_table(pivot, index='year',columns='month',values='Area Harvested')
    table_area = pd.DataFrame(data=table_area, index=table_area.index, columns=[1,3,7,9,12])
    table_area[12] = table_area[table_area.columns[:-1]].sum(1)
    table_prod = pd.pivot_table(pivot, index='year',columns='month',values='Quantity Produced')
    table_prod = pd.DataFrame(data=table_prod, index=table_prod.index, columns=[1,3,7,9,12])
    table_prod[12] = table_prod[table_prod.columns[:-1]].sum(1)
    # MultiIndex
    mdx = CreateMultiIndex(fnid, name)
    table_area.columns = mdx
    table_prod.columns = mdx
    frame_area.append(table_area)
    frame_prod.append(table_prod)
area = pd.concat(frame_area, axis=1)
prod = pd.concat(frame_prod, axis=1)
# Reindex year
area = area.reindex(np.arange(area.index.min(), 2020))
prod = prod.reindex(np.arange(prod.index.min(), 2020))

# Correct crop reporting units 
# Summation Afmadow (SO1990R22805) and Hagar (SO1990R22806) to represent Afmadow (SO1990A22802)
area_temp = area['SO1990R22805'].add(area['SO1990R22806'], fill_value=0)
prod_temp = prod['SO1990R22805'].add(prod['SO1990R22806'], fill_value=0)
area['SO1990A22802'] = area['SO1990A22802'].add(area_temp, fill_value=0)
prod['SO1990A22802'] = prod['SO1990A22802'].add(prod_temp, fill_value=0)
area = area.drop(['SO1990R22805', 'SO1990R22806'], axis=1)
prod = prod.drop(['SO1990R22805', 'SO1990R22806'], axis=1)

### (3) Manual removing unrealistic records
In Admin-2 level data, we found unrealistic records. So, here we manually remove those records including:
- 26 districts have the exactly same `1 mt/ha` yield values in `Deyr` season of `2001`.
- Three districts have unrealistic yield values in `Gu` season of `2005`: Bu'aale (11.65 mt/ha), Jilib (2.27 mt/ha), and Saakow (20.8 mt/ha).

In [10]:
# Remove unrealistic values in 2001
crop = prod/area
temp = crop.loc[2001,pd.IndexSlice[:,:,'Deyr',:]]
remove_index = temp.index[temp == 1]
prod.loc[2001,remove_index] = np.nan
area.loc[2001,remove_index] = np.nan
# Remove unrealistic values in 2005
area.loc[2005,pd.IndexSlice[:,["Bu'aale","Jilib","Saakow"],'Gu',:]] = np.nan
prod.loc[2005,pd.IndexSlice[:,["Bu'aale","Jilib","Saakow"],'Gu',:]] = np.nan

# Recalculate annual total area and production
for fnid in area.columns.get_level_values(0).unique():
    temp = area.loc[:, pd.IndexSlice[fnid,:,['Deyr','Deyr-off','Gu','Gu-off'],:]]
    resum = temp.sum(1); resum[temp.isna().sum(1) == 4] = np.nan
    area.loc[:, pd.IndexSlice[fnid,:,'Annual',:]] = resum.values
    temp = prod.loc[:, pd.IndexSlice[fnid,:,['Deyr','Deyr-off','Gu','Gu-off'],:]]
    resum = temp.sum(1); resum[temp.isna().sum(1) == 4] = np.nan
    prod.loc[:, pd.IndexSlice[fnid,:,'Annual',:]] = resum.values

### (4) Aggregate Admin-2 level data to Admin-1 level
In the above section, we made the area and production tables for all available Admin 1 and 2 levels. In this section, we merge Admin-2 level data to reconstruct Admin-1 level with including the existing Admin-1 level data. Then we will split Admin-2 level data from the entire tables.

In [11]:
fnid_data = area.columns.get_level_values(0).unique()
area_frame = []
prod_frame = []
# Loop per FNID_ADM1
for i, (fnid, name) in SO_adm1_1990[['FNID', 'ADMIN1']].iterrows():
    # - Merge avaliable data
    aval = fnid_data[fnid_data.isin([fnid, *adm_joint[fnid]])]
    if len(aval) == 0: continue 
    aval_area_frame = []
    aval_prod_frame = []
    for i in aval:
        temp_area = area[i]
        temp_prod = prod[i]
        temp_area.columns = temp_area.columns.droplevel([0])
        temp_prod.columns = temp_prod.columns.droplevel([0])
        aval_area_frame.append(temp_area)
        aval_prod_frame.append(temp_prod)
    merged_area = reduce(lambda  x, y: x.add(y, fill_value=0), aval_area_frame)
    merged_prod = reduce(lambda  x, y: x.add(y, fill_value=0), aval_prod_frame)
    merged_area.columns = CreateMultiIndex(fnid, name)
    merged_prod.columns = CreateMultiIndex(fnid, name)
    area_frame.append(merged_area)
    prod_frame.append(merged_prod)
area1 = pd.concat(area_frame, axis=1)
prod1 = pd.concat(prod_frame, axis=1)
# Finally select Admin-2 level from the entire pool
fnid_data_adm2 = fnid_data[~fnid_data.isin(adm_joint.keys())]
area2 = area[fnid_data_adm2]
prod2 = prod[fnid_data_adm2]

### (5) Finalize yield data

In [12]:
# Calculate yield
crop1 = prod1 / area1
crop2 = prod2 / area2

# Calculate national yield (total production / total planted area) in both seasons and annual
nmiss_default = len(ADM1_LIST) - len(area1.columns.get_level_values(0).unique())
ntnl_annual = pd.DataFrame(index = area1.index, columns = ['Area','Production','Yield','Miss_districts'])
ntnl_deyr = ntnl_annual.copy()
ntnl_gu = ntnl_annual.copy()
for year in ntnl_annual.index:
    # Annual
    ntnl_annual.loc[year, 'Area'] = area1.loc[year,pd.IndexSlice[:,:,'Annual',:]].sum()
    ntnl_annual.loc[year, 'Production'] = prod1.loc[year,pd.IndexSlice[:,:,'Annual',:]].sum()
    nmiss = area1.loc[year,pd.IndexSlice[:,:,'Annual',:]].isna().sum()
    ntnl_annual.loc[year, 'Miss_districts'] = (nmiss + nmiss_default)/len(ADM1_LIST)*100
    # Deyr season
    ntnl_deyr.loc[year, 'Area'] = area1.loc[year,pd.IndexSlice[:,:,'Deyr',:]].sum()
    ntnl_deyr.loc[year, 'Production'] = prod1.loc[year,pd.IndexSlice[:,:,'Deyr',:]].sum()
    nmiss = area1.loc[year,pd.IndexSlice[:,:,'Deyr',:]].isna().sum()
    ntnl_deyr.loc[year, 'Miss_districts'] = (nmiss + nmiss_default)/len(ADM1_LIST)*100
    # Gu season
    ntnl_gu.loc[year, 'Area'] = area1.loc[year,pd.IndexSlice[:,:,'Gu',:]].sum()
    ntnl_gu.loc[year, 'Production'] = prod1.loc[year,pd.IndexSlice[:,:,'Gu',:]].sum()
    nmiss = area1.loc[year,pd.IndexSlice[:,:,'Gu',:]].isna().sum()
    ntnl_gu.loc[year, 'Miss_districts'] = (nmiss + nmiss_default)/len(ADM1_LIST)*100
ntnl_annual['Yield'] = ntnl_annual['Production'] / ntnl_annual['Area']
ntnl_deyr['Yield'] = ntnl_deyr['Production'] / ntnl_deyr['Area']
ntnl_gu['Yield'] = ntnl_gu['Production'] / ntnl_gu['Area']
ntnl_annual[ntnl_annual == 0] = np.nan
ntnl_deyr[ntnl_deyr == 0] = np.nan
ntnl_gu[ntnl_gu == 0] = np.nan

# Annual and seasonal (districts and national)
yield1a = crop1.loc[:,pd.IndexSlice[:,:,'Annual',:]]; yield1a.columns = yield1a.columns.droplevel([1,2,3])
area1a = area1.loc[:,pd.IndexSlice[:,:,'Annual',:]]; area1a.columns = area1a.columns.droplevel([1,2,3])
prod1a = prod1.loc[:,pd.IndexSlice[:,:,'Annual',:]]; prod1a.columns = prod1a.columns.droplevel([1,2,3])
yield1s = crop1.loc[:,pd.IndexSlice[:,:,'Deyr',:]]; yield1s.columns = yield1s.columns.droplevel([1,2,3])
area1s = area1.loc[:,pd.IndexSlice[:,:,'Deyr',:]]; area1s.columns = area1s.columns.droplevel([1,2,3])
prod1s = prod1.loc[:,pd.IndexSlice[:,:,'Deyr',:]]; prod1s.columns = prod1s.columns.droplevel([1,2,3])
yield1l = crop1.loc[:,pd.IndexSlice[:,:,'Gu',:]]; yield1l.columns = yield1l.columns.droplevel([1,2,3])
area1l = area1.loc[:,pd.IndexSlice[:,:,'Gu',:]]; area1l.columns = area1l.columns.droplevel([1,2,3])
prod1l = prod1.loc[:,pd.IndexSlice[:,:,'Gu',:]]; prod1l.columns = prod1l.columns.droplevel([1,2,3])

# Export to HDF files
# National
save_hdf('./data/crop/SO_maize_adm0_yield_annual.hdf', ntnl_annual)
save_hdf('./data/crop/SO_maize_adm0_yield_deyr.hdf', ntnl_deyr)
save_hdf('./data/crop/SO_maize_adm0_yield_gu.hdf', ntnl_gu)
# Admin-1 level
save_hdf('./data/crop/SO_maize_adm1_yield_annual.hdf', yield1a)
save_hdf('./data/crop/SO_maize_adm1_yield_deyr.hdf', yield1s)
save_hdf('./data/crop/SO_maize_adm1_yield_gu.hdf', yield1l)
save_hdf('./data/crop/SO_maize_adm1_area_annual.hdf', area1a)
save_hdf('./data/crop/SO_maize_adm1_area_deyr.hdf', area1s)
save_hdf('./data/crop/SO_maize_adm1_area_gu.hdf', area1l)
save_hdf('./data/crop/SO_maize_adm1_prod_annual.hdf', prod1a)
save_hdf('./data/crop/SO_maize_adm1_prod_deyr.hdf', prod1s)
save_hdf('./data/crop/SO_maize_adm1_prod_gu.hdf', prod1l)

# Export to Excel file
fn_out = './data/crop/SO_maize_summary.xlsx'
with pd.ExcelWriter(fn_out, engine='xlsxwriter') as writer:
    # National
    ntnl_annual.to_excel(writer, sheet_name='adm0_yield_annual')
    ntnl_deyr.to_excel(writer, sheet_name='adm0_yield_deyr')
    ntnl_gu.to_excel(writer, sheet_name='adm0_yield_gu')
    # Admin-1 level
    yield1a.to_excel(writer, sheet_name='adm1_yield_annual')
    area1a.to_excel(writer, sheet_name='adm1_area_annual')
    prod1a.to_excel(writer, sheet_name='adm1_prod_annual')
    yield1s.to_excel(writer, sheet_name='adm1_yield_deyr')
    area1s.to_excel(writer, sheet_name='adm1_area_deyr')
    prod1s.to_excel(writer, sheet_name='adm1_prod_deyr')
    yield1l.to_excel(writer, sheet_name='adm1_yield_gu')
    area1l.to_excel(writer, sheet_name='adm1_area_gu')
    prod1l.to_excel(writer, sheet_name='adm1_prod_gu')
    print('%s is saved..' % fn_out)

./data/crop/SO_maize_adm0_yield_annual.hdf is saved.
./data/crop/SO_maize_adm0_yield_deyr.hdf is saved.
./data/crop/SO_maize_adm0_yield_gu.hdf is saved.
./data/crop/SO_maize_adm1_yield_annual.hdf is saved.
./data/crop/SO_maize_adm1_yield_deyr.hdf is saved.
./data/crop/SO_maize_adm1_yield_gu.hdf is saved.
./data/crop/SO_maize_adm1_area_annual.hdf is saved.
./data/crop/SO_maize_adm1_area_deyr.hdf is saved.
./data/crop/SO_maize_adm1_area_gu.hdf is saved.
./data/crop/SO_maize_adm1_prod_annual.hdf is saved.
./data/crop/SO_maize_adm1_prod_deyr.hdf is saved.
./data/crop/SO_maize_adm1_prod_gu.hdf is saved.
./data/crop/SO_maize_summary.xlsx is saved..


## 3) Complete dataset of Maize yield for East Africa

### (1) Merge shapefiles of Kenya and Somalia

In [13]:
# Merge shapefiles
KE_adm1_2013 = gpd.read_file('./data/shapefile/KE_Admin1_2013.shp')
SO_adm1_1990 = gpd.read_file('./data/shapefile/SO_Admin1_1990.shp')
EA_adm1 = pd.concat([KE_adm1_2013, SO_adm1_1990], axis=0)[['FNID', 'ADMIN0', 'ADMIN1', 'geometry']]
fn_out = './data/shapefile/EA_admin1.shp'
EA_adm1.to_file(fn_out)
print('%s is saved.' % fn_out)

./data/shapefile/EA_admin1.shp is saved.
